In [1]:
import csv
from math import sin, cos, sqrt, atan2, radians, asin, pi


# we use class node to create nodes using their names and their connections
class Node:
    def __init__(self,name):
        self.name = name
        self.node_neighbors = []
        
    def connection(self,node,weight):
        self.node_neighbors.append((node.name,weight))


# we use class edge to create edges 
class Edge:
    def __init__(self,frm,to,weight=0,directed=False):
        self.frm = frm
        self.to = to
        self.weight = weight
        self.directed = directed


# we use class graph to construct our graph using edges from our csv file by creating nodes and connections
class Graph:
    def __init__(self,edges):
        self.nodes = {}
        self.edges ={}
        for frm, to , weight, directed in edges:
            node_from = Node(frm)
            node_to = Node(to)
            edge = Edge(frm,to,weight,directed)
            key = (frm,to)
            self.edges[key] = edge
            if node_from.name not in self.nodes:
                self.nodes[node_from.name] = node_from
                
            if node_to.name not in self.nodes:
                self.nodes[node_to.name] = node_to
           
            self.nodes[node_from.name].connection(node_to,weight)
            if directed =="FALSE":
                self.nodes[node_to.name].connection(node_from,weight)


# we use this method to represent our graph as adjacency list graph representation
    def __repr__ (self):
        result = " "
        for node in self.nodes:
            result += "{} : {}\n".format(node,self.nodes[node].node_neighbors)
        return result


# here we implement A * shortest path algorithm to find degree centrality
def A_star_path_finding_degree(graph,heuristic_distance, initial_node):
    total_node = len(graph.nodes.keys())
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    degree_dict = {}

    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index
        
    queue = [(0+heuristic_distance[initial_node], initial_node)]
    index = index_dict[initial_node]
    distance[index] = 0
    
    while(queue):
        degree = 0
        queue.sort(reverse=False)
        current_weight_heuristic_value,current_node = queue.pop(0)
        
        degree_dict[current_node] = degree

        for value, weight in graph.nodes[current_node].node_neighbors:
            degree_dict[current_node] +=1
            index = index_dict[value]
            newWeight = current_weight_heuristic_value-heuristic_distance[current_node] + int(weight)
            
            if newWeight <= distance[index]:
                distance[index] = newWeight 
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue.pop(index)
                        newWeight += heuristic_distance[value]
                        queue.append((newWeight,j))
                        
                if (newWeight,value) not in queue:
                    newWeight += heuristic_distance[value]
                    queue.append((newWeight,value))

        degree_dict[current_node] =round(degree_dict[current_node]/(total_node-1), 3)
        if current_node == initial_node:
            return degree_dict[current_node]

    raise ValueError("Target city node not in city node list!")

# here we implement A * shortest path algorithm to find closeness centrality
def A_star_path_finding_closeness(graph,heuristic_distance, initial_node,target_node):
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}

    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index
        
    queue = [(0+heuristic_distance[initial_node], initial_node)]
    index = index_dict[initial_node]
    distance[index] = 0

    while(queue):
        queue.sort(reverse=False)
        current_weight_heuristic_value,current_node = queue.pop(0)

        if current_node == target_node:
                    return distance[index_dict[current_node]]

        for value, weight in graph.nodes[current_node].node_neighbors:
            
            index = index_dict[value]
            newWeight = current_weight_heuristic_value-heuristic_distance[current_node] + int(weight)
            
            if newWeight <= distance[index]:
                distance[index] = newWeight 
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue.pop(index)
                        newWeight += heuristic_distance[value]
                        queue.append((newWeight,j))
                        
                if (newWeight,value) not in queue:
                    newWeight += heuristic_distance[value]
                    queue.append((newWeight,value))

    return distance[index_dict[current_node]]


# implementation of Dijkstra's algorithm to find degree centrality
def Dijkstra_shortest_path_degree(graph,initial_node):
    total_node = len(graph.nodes.keys())
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    degree_dict = {}

    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index

    queue = []
    queue.append((0,initial_node))
    index = index_dict[initial_node]
    
    while(queue):
        degree = 0
        queue.sort(reverse=False)
        current_weight, current_node = queue.pop(0)
        degree_dict[current_node] = degree
        for value, weight in graph.nodes[current_node].node_neighbors:
            degree_dict[current_node] +=1
            index = index_dict[value]
            newWeight = current_weight + int(weight)
            if newWeight <= distance[index]:
                distance[index] = newWeight
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue[index] = (newWeight,j)

                if (newWeight,value) not in queue:
                    queue.append((newWeight,value))

        degree_dict[current_node] = round(degree_dict[current_node]/(total_node-1), 3)
        if current_node == initial_node:
            return degree_dict[current_node]
        
    raise ValueError("Target city node not in city node list!") 


# here we implement Dijkstra's algorithm to calculate closeness centrality           
def Dijkstra_shortest_path_closeness(graph,initial_node,target_node):
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    
    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index

    queue = []
    queue.append((0,initial_node))
    index = index_dict[initial_node]
    distance[index] = 0

    while(queue):
        queue.sort(reverse=False)
        current_weight, current_node = queue.pop(0)
   
        if current_node == target_node:
                    return distance[index_dict[current_node]]

        for value, weight in graph.nodes[current_node].node_neighbors:
            index = index_dict[value]
            newWeight = current_weight + int(weight)
            if newWeight <= distance[index]:
                distance[index] = newWeight
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue[index] = (newWeight,j)

                if (newWeight,value) not in queue:
                    queue.append((newWeight,value))

    return distance[index_dict[current_node]]   


# here we implement Dijkstra's algorithm to calculate betweenness centrality           
def Dijkstra_shortest_path_betweenness(graph,initial_node,target_node,base_node):
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    
    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index

    queue = []
    queue.append((0,initial_node))
    index = index_dict[initial_node]
    distance[index] = 0
    list_shortest_path = []
    path_length = 0

    while(queue):
        queue.sort(reverse=False)
        current_weight, current_node = queue.pop(0)
        list_shortest_path.append((current_node,current_weight))
        if current_node == target_node:
            path_length = distance[index_dict[current_node]]
            break
        for value, weight in graph.nodes[current_node].node_neighbors:
            index = index_dict[value]
            newWeight = current_weight + int(weight)
            if newWeight <= distance[index]:
                distance[index] = newWeight
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue[index] = (newWeight,j)

                if (newWeight,value) not in queue:
                    queue.append((newWeight,value))

    shortest_path_nodes = []
    temp = []
    index = 0
    destination_node, shortest_path = list_shortest_path.pop()
    shortest_path_nodes.append(destination_node)
    for _ in range(len(list_shortest_path)):
        node , node_distance = list_shortest_path.pop()

        while(temp):
            prev_node , prev_distance = temp[index]
            for value , weight in graph.nodes[prev_node].node_neighbors:
                if node == value:
                    if int(weight)+node_distance == prev_distance:
                        index+=1
                        shortest_path_nodes.append((node))
                        temp.append((node,node_distance))

            break

        for value ,weight in graph.nodes[node].node_neighbors:
            if destination_node == value:
                if (node_distance + int(weight)) == shortest_path:
                    temp.append((node,node_distance))
                    shortest_path_nodes.append(node)
    
    count = 0
    shortest_path_nodes.reverse()   
    index = 0
    min_index = 3

    if len(shortest_path_nodes) > min_index:
        for _ in range(len(shortest_path_nodes)):
            for value, weight in graph.nodes[shortest_path_nodes[index]].node_neighbors:
                if shortest_path_nodes[index+2] == value:
                    count+=2

            index +=1
            if index+2 == len(shortest_path_nodes):
                break

    if count == 0:
        count = 1

    if base_node in shortest_path_nodes:
        return path_length,count
    else:
        return 0,count

# here we implement A * shortest path algorithm  to calculate the betweenness_centrality
def A_star_path_finding_betweenness(graph,heuristic_distance, initial_node,target_node,base_node):
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    
    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index
        
    queue = [(0+heuristic_distance[initial_node], initial_node)]
    index = index_dict[initial_node]
    distance[index] = 0
    list_shortest_path = []
    path_length = 0

    while(queue):
        queue.sort(reverse=False)
        current_weight_heuristic_value,current_node = queue.pop(0)
        list_shortest_path.append((current_node,round(current_weight_heuristic_value-heuristic_distance[current_node],1)))
        if current_node == target_node:
            path_length = distance[index_dict[current_node]]
            break

        for value, weight in graph.nodes[current_node].node_neighbors:
            index = index_dict[value]
            newWeight = current_weight_heuristic_value-heuristic_distance[current_node] + int(weight)
            
            if newWeight <= distance[index]:
                distance[index] = newWeight 
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue.pop(index)
                        newWeight += heuristic_distance[value]
                        queue.append((newWeight,j))
    
                if (newWeight,value) not in queue:
                    newWeight += heuristic_distance[value]
                    queue.append((newWeight,value))

    shortest_path_nodes = []
    temp = []
    index = 0
    destination_node, shortest_path = list_shortest_path.pop()
    shortest_path_nodes.append(destination_node)

    for _ in range(len(list_shortest_path)):
        node , node_distance = list_shortest_path.pop()

        while(temp):
            prev_node , prev_distance = temp[index]
            for value , weight in graph.nodes[prev_node].node_neighbors:
                if node == value:
                    if int(weight)+node_distance == prev_distance:
                        index+=1
                        shortest_path_nodes.append((node))
                        temp.append((node,node_distance))

            break

        for value ,weight in graph.nodes[node].node_neighbors:
            if destination_node == value:
                if (node_distance + int(weight)) == shortest_path:
                    temp.append((node,node_distance))
                    shortest_path_nodes.append(node)

    count = 0
    shortest_path_nodes.reverse()   
    index = 0
    min_index = 3
    if len(shortest_path_nodes) > min_index:
        for _ in range(len(shortest_path_nodes)):
            for value, weight in graph.nodes[shortest_path_nodes[index]].node_neighbors:
                if shortest_path_nodes[index+2] == value:
                    count+=2

            index +=1
            if index+2 == len(shortest_path_nodes):
                break

    if count == 0:
        count = 1

    if base_node in shortest_path_nodes:
        return path_length,count
    else:
        return 0,count


# here goes the implementation for reading csv file of edge list's
file = open('City_Edges_list.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
edges = []
# we add all the edges list into edges dictionary
for edge in csvreader:
        edges.append(tuple(edge))


# here goes the implementation for reading csv file of each city/node latitude and longitude       
file = open('cities_latitude_longitudes.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
lat_long_list = {}
# we add all the latitude and longitude as a tuple in to lat_long_list
for city,latitude,longitude in csvreader:
    lat_long_list[city] = (float(latitude),float(longitude))


# we use this function to calculate the distance between any to nodes using latitude and longitude points
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    
    return 12742 * asin(sqrt(a))



# this function is used to calculate the heuristic value for each nodes using latitude and longitude
def heuristic_value_calculate(lat_long,graph,target_node):
    heuristic_distance = {}
    for _ in range(len(list(lat_long.keys()))):
        node = list(lat_long.keys())[_]
        if node == target_node:
            heuristic_distance[target_node] = float('inf')
        value = distance(lat_long[node][0],lat_long[node][1],lat_long[target_node][0],lat_long[target_node][1])
        heuristic_distance[node] = value

    return heuristic_distance

# this function is used to sum the shortest distances between given initial node and all the reset nodes
def shortest_distance_value_A_star(search_algorithm,graph,initial_node):
    total_distance = 0

    for target_node in list(graph.nodes.keys()):
        heuristic_distance = heuristic_value_calculate(lat_long_list,graph,target_node)
        total_distance+=search_algorithm(graph,heuristic_distance,initial_node,target_node)

    return total_distance

# this function is used to sum the shortest distances between given initial node and all the reset nodes
def shortest_distance_value(search_algorithm,graph,initial_node):
    total_distance = 0

    for target_node in list(graph.nodes.keys()):
        total_distance+=search_algorithm(graph,initial_node,target_node)

    return total_distance


# create our graph instance
graph = Graph(edges)
graph



def DJ_closeness_centrality(graph,initial_node):
    total_nodes = len(graph.nodes.keys())
    distance = shortest_distance_value(Dijkstra_shortest_path_closeness,graph,initial_node)
    closeness_cent = (total_nodes-1)/distance

    return round(closeness_cent,7)


# print(DJ_closeness_centrality(graph, "Neamt"))

def Ast_closeness_centrality(graph,initial_node):
    total_nodes = len(graph.nodes.keys())
    distance = shortest_distance_value_A_star(A_star_path_finding_closeness,graph,initial_node)
    closeness_cent = (total_nodes-1)/distance
   
    return round(closeness_cent ,7)

# print(Ast_closeness_centrality(graph,"Neamt"))

def DJ_betweenness_centrality(graph,base_node):
    betweenness_centrality = 0
    temp = 0
    for initial_node in graph.nodes.keys():
        for target_node in graph.nodes.keys():
            if base_node == initial_node or base_node == target_node:
                break
            distance , num_path = Dijkstra_shortest_path_betweenness(graph,initial_node,target_node,base_node)
            betweenness_centrality += distance/num_path
    return betweenness_centrality

# print(DJ_betweenness_centrality(graph,"Iasi"))

def Ast_betweenness_centrality(graph,base_node):
    betweenness_centrality = 0
    temp = 0
    for initial_node in graph.nodes.keys():
        for target_node in graph.nodes.keys():
            if base_node == initial_node or base_node == target_node:
                break
            heuristic_distance = heuristic_value_calculate(lat_long_list,graph,target_node)
            distance , num_path = A_star_path_finding_betweenness(graph,heuristic_distance,initial_node,target_node,base_node)
            betweenness_centrality += distance/num_path
    return betweenness_centrality

# Ast_betweenness_centrality(graph,"Iasi")
